<a href="https://colab.research.google.com/github/PradipNichite/Youtube-Tutorials/blob/main/Youtube_NL2SQL_llama_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Realted Videos

LangChain, SQL Agents & OpenAI LLMs: Query Database Using Natural Language https://youtu.be/VG9KYCS0-8E?si=EAqu-9DS6QX9eitR

Mastering LlamaIndex : Create, Save & Load Indexes, Customize LLMs, Prompts & Embeddings: https://youtu.be/XGBQ_f-Yy48?si=ZWs0meSlQpLPcCE0

In [1]:
!pip install llama-index pymysql -q

In [2]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO, force=True)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
from IPython.display import Markdown, display

https://www.mysqltutorial.org/mysql-sample-database.aspx

In [4]:
db_user = "admin"
db_password = "llama_nl2sql"
db_host = "langchainsql.cl0j8hicdoox.us-east-1.rds.amazonaws.com"
db_name = "classicmodels"

In [5]:
from sqlalchemy import create_engine, text

# Construct the connection string
connection_string = f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}"

# Create an engine instance
engine = create_engine(connection_string)

# Test the connection using raw SQL
with engine.connect() as connection:
    result = connection.execute(text("select * from customers limit 3"))
    for row in result:
        print(row)

(103, 'Atelier graphique', 'Schmitt', 'Carine ', '40.32.2555', '54, rue Royale', None, 'Nantes', None, '44000', 'France', 1370, Decimal('21000.00'))
(112, 'Signal Gift Stores', 'King', 'Jean', '7025551838', '8489 Strong St.', None, 'Las Vegas', 'NV', '83030', 'USA', 1166, Decimal('71800.00'))
(114, 'Australian Collectors, Co.', 'Ferguson', 'Peter', '03 9520 4555', '636 St Kilda Road', 'Level 3', 'Melbourne', 'Victoria', '3004', 'Australia', 1611, Decimal('117300.00'))


In [30]:
table_details = {
    "customers": "stores customer’s data.",
    "products": "stores a list of scale model cars.",
    "productlines": "stores a list of product line categories.",
    "orders": "stores sales orders placed by customers.",
    "orderdetails": "stores sales order line items for each sales order.",
    "payments": "stores payments made by customers based on their accounts.",
    "employees": "stores all employee information as well as the organization structure such as who reports to whom.",
    "offices": "stores sales office data."
}


https://gpt-index.readthedocs.io/en/latest/api_reference/struct_store.html#llama_index.utilities.sql_wrapper.SQLDatabase

In [6]:
from llama_index import SQLDatabase
tables = ["customers","orders"]
# sql_database = SQLDatabase(engine, include_tables=tables,sample_rows_in_table_info=5)
sql_database = SQLDatabase(engine, sample_rows_in_table_info=2)
sql_database

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
NumExpr defaulting to 2 threads.


In [7]:
list(sql_database._all_tables)

['payments',
 'employees',
 'customers',
 'orders',
 'productlines',
 'products',
 'orderdetails',
 'offices']

In [8]:
print(sql_database.table_info)


CREATE TABLE customers (
	`customerNumber` INTEGER NOT NULL, 
	`customerName` VARCHAR(50) NOT NULL, 
	`contactLastName` VARCHAR(50) NOT NULL, 
	`contactFirstName` VARCHAR(50) NOT NULL, 
	phone VARCHAR(50) NOT NULL, 
	`addressLine1` VARCHAR(50) NOT NULL, 
	`addressLine2` VARCHAR(50), 
	city VARCHAR(50) NOT NULL, 
	state VARCHAR(50), 
	`postalCode` VARCHAR(15), 
	country VARCHAR(50) NOT NULL, 
	`salesRepEmployeeNumber` INTEGER, 
	`creditLimit` DECIMAL(10, 2), 
	PRIMARY KEY (`customerNumber`), 
	CONSTRAINT customers_ibfk_1 FOREIGN KEY(`salesRepEmployeeNumber`) REFERENCES employees (`employeeNumber`)
)DEFAULT CHARSET=latin1 ENGINE=InnoDB

/*
2 rows from customers table:
customerNumber	customerName	contactLastName	contactFirstName	phone	addressLine1	addressLine2	city	state	postalCode	country	salesRepEmployeeNumber	creditLimit
103	Atelier graphique	Schmitt	Carine 	40.32.2555	54, rue Royale	None	Nantes	None	44000	France	1370	21000.00
112	Signal Gift Stores	King	Jean	7025551838	8489 Strong St

In [9]:
import os
import openai
os.environ["OPENAI_API_KEY"] = "sk-bE"
openai.api_key = os.environ["OPENAI_API_KEY"]

https://gpt-index.readthedocs.io/en/latest/examples/callbacks/TokenCountingHandler.html

In [12]:
import tiktoken
from llama_index.callbacks import CallbackManager, TokenCountingHandler
token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model("gpt-3.5-turbo").encode
)

callback_manager = CallbackManager([token_counter])

In [13]:
from llama_index import ServiceContext, LLMPredictor, OpenAIEmbedding, PromptHelper
from llama_index.llms import OpenAI
from llama_index import ServiceContext
llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo")

service_context = ServiceContext.from_defaults(
  llm=llm,callback_manager=callback_manager
)

In [14]:
from llama_index.indices.struct_store.sql_query import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    service_context=service_context
)

In [25]:
# query_str = "Which customer has the highest orders?"
query_str = "when was order number 10100 shiiped?"
# query_str ="What are the most frequently mentioned keywords or phrases in the comments made by sales representatives"
response = query_engine.query(query_str)

INFO:llama_index.indices.struct_store.sql_query:> Table desc str: Table 'customers' has columns: customerNumber (INTEGER), customerName (VARCHAR(50)), contactLastName (VARCHAR(50)), contactFirstName (VARCHAR(50)), phone (VARCHAR(50)), addressLine1 (VARCHAR(50)), addressLine2 (VARCHAR(50)), city (VARCHAR(50)), state (VARCHAR(50)), postalCode (VARCHAR(15)), country (VARCHAR(50)), salesRepEmployeeNumber (INTEGER), creditLimit (DECIMAL(10, 2)), and foreign keys: ['salesRepEmployeeNumber'] -> employees.['employeeNumber'].

Table 'employees' has columns: employeeNumber (INTEGER), lastName (VARCHAR(50)), firstName (VARCHAR(50)), extension (VARCHAR(10)), email (VARCHAR(100)), officeCode (VARCHAR(10)), reportsTo (INTEGER), jobTitle (VARCHAR(50)), and foreign keys: ['reportsTo'] -> employees.['employeeNumber'], ['officeCode'] -> offices.['officeCode'].

Table 'offices' has columns: officeCode (VARCHAR(10)), city (VARCHAR(50)), phone (VARCHAR(50)), addressLine1 (VARCHAR(50)), addressLine2 (VARCHA

In [20]:
print(response.response)

Order number 10100 was shipped on January 10, 2003.


In [21]:
print(response.metadata['sql_query'])

SELECT shippedDate
FROM orders
WHERE orderNumber = 10100


In [22]:
print(response.metadata['result'])

[(datetime.date(2003, 1, 10),)]


In [26]:
print(token_counter.total_llm_token_count)

904


In [27]:
token_counter.reset_counts()

In [31]:
from llama_index.objects import ObjectIndex
from llama_index.objects import SQLTableNodeMapping, SQLTableSchema
import pandas as pd

tables = list(sql_database._all_tables)
table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = []
for table in tables:
    table_schema_objs.append((SQLTableSchema(table_name = table, context_str = table_details[table])))

In [32]:
table_schema_objs

[SQLTableSchema(table_name='payments', context_str='stores payments made by customers based on their accounts.'),
 SQLTableSchema(table_name='employees', context_str='stores all employee information as well as the organization structure such as who reports to whom.'),
 SQLTableSchema(table_name='customers', context_str='stores customer’s data.'),
 SQLTableSchema(table_name='orders', context_str='stores sales orders placed by customers.'),
 SQLTableSchema(table_name='productlines', context_str='stores a list of product line categories.'),
 SQLTableSchema(table_name='products', context_str='stores a list of scale model cars.'),
 SQLTableSchema(table_name='orderdetails', context_str='stores sales order line items for each sales order.'),
 SQLTableSchema(table_name='offices', context_str='stores sales office data.')]

In [33]:
from llama_index.indices.struct_store import SQLTableRetrieverQueryEngine
from llama_index import VectorStoreIndex


obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
    service_context=service_context
)

In [34]:
query_engine = SQLTableRetrieverQueryEngine(
    sql_database, obj_index.as_retriever(similarity_top_k=3), service_context=service_context
)
response = query_engine.query("How many customers we have?")

INFO:llama_index.indices.struct_store.sql_query:> Table desc str: Table 'customers' has columns: customerNumber (INTEGER), customerName (VARCHAR(50)), contactLastName (VARCHAR(50)), contactFirstName (VARCHAR(50)), phone (VARCHAR(50)), addressLine1 (VARCHAR(50)), addressLine2 (VARCHAR(50)), city (VARCHAR(50)), state (VARCHAR(50)), postalCode (VARCHAR(15)), country (VARCHAR(50)), salesRepEmployeeNumber (INTEGER), creditLimit (DECIMAL(10, 2)), and foreign keys: ['salesRepEmployeeNumber'] -> employees.['employeeNumber']. The table description is: stores customer’s data.

Table 'orders' has columns: orderNumber (INTEGER), orderDate (DATE), requiredDate (DATE), shippedDate (DATE), status (VARCHAR(15)), comments (TEXT), customerNumber (INTEGER), and foreign keys: ['customerNumber'] -> customers.['customerNumber']. The table description is: stores sales orders placed by customers.

Table 'payments' has columns: customerNumber (INTEGER), checkNumber (VARCHAR(50)), paymentDate (DATE), amount (DE

In [35]:
print(response)

We have 122 customers.


In [36]:
print(token_counter.total_llm_token_count)

514
